## OED Data Selection

Notebook that retrieves relevant data for OED API.

For relevant documentation, look [here](https://languages.oup.com/research/oed-researcher-api/#see-documentation)

In [1]:
import requests
import pickle
import pandas as pd
from tqdm.notebook import tqdm

/Users/Giorgia/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
auth = pickle.load(open('./dev/oed_keys.pkcl',"rb"))

FileNotFoundError: [Errno 2] No such file or directory: './dev/oed_keys.pkcl'

In [ ]:
def query_oed(
          auth:dict,
          endpoint:str,
          query:str,
          flags:str='',
          level:str='',
          verbose=True):
    
    """
    Get data from Oxford English Dictionary.
    Function requires an endpoint _and_ query as arguments. 
    
    Arguments:
    
        auth (dict): a dictionary with authentication inforamtion, needs details for 'app_id' and 'app_key'
    
        endpoint (str): select which endpoint to query, examples are word, sense, semanticclass etc
        
        query (str): query for the specific endpoint, most often a specific id, such as 'machine_nn01' or '120172'
        
        flags (str): options appended to query to include, for example, quotations instead of quotation ids
                     example "include_senses=false&include_quotations=false"
        
        level (str): at which level to query the endpoint, 
                     e.g. get sense of the query word, get siblings for semantic class etc
                     standard value is empty string
        
        verbose (bool): print the URL used for retrieving information from the API
        
    Returns:
    
        JSON of the response
        
    Example uses:
    
        query_oed(auth, 'word', 'machine_nn01')
            -> Retrieves information for the word machine_nn01.
        
        query_oed(auth, 'word','machine_nn01',level='quotations')
            -> Retrieves all quotations for machine_nn01.
        
        query_oed(auth, 'word', 'machine_nn01', flags="include_senses=true&include_quotations=true")
            -> Retrieves all senses and quotations  for the word machine_nn01.
            
        query_oed(auth, 'semanticclass', '163378')
            -> Retrieves semantic class with id 163378.
            
        query_oed(auth, 'semanticclass', '163378', level='children')
            -> Retrieves all children for the semanticlass with id 163378.
            
        query_oed(auth,'semanticclass', '163378', level='branchsenses',flags="current_in='1750-1950'")
            -> get all senses (siblings _and_ descendants) branching out from semantic class with id 163378
               restrict query to all senses observed between 1750 and 1950.
                
    """
    
    base_url = "https://oed-researcher-api.oxfordlanguages.com/oed/api/v0.2"
    
    url = f"{base_url}/{endpoint}/{query}" # build url
    
    if level: # if a level has been specified add this to the url
        url = f"{url}/{level}/"
    
    if flags: #  add flag to url with a question mark
        url = f"{url}?{flags}"
        
    response = requests.get(url, headers=auth) 
    
    if verbose:
        print(url)
        
    if response.status_code == 200: # check status code 
        return response.json() # return the data as json
    
    else:
        raise Exception(f"Error while accessing the API\nResponse code={response.status_code}")
    


# Retrieve and save all senses and quotations

In [3]:
machine = query_oed(auth, 'word','machine_nn01',flags="include_senses=true&include_quotations=true")

NameError: name 'query_oed' is not defined

In [ ]:
# with open('./dev/machine_senses_quotations.pckl','wb') as out_pickle:
#     pickle.dump(machine,out_pickle)

In [ ]:
with open('./dev/machine_senses_quotations.pckl','rb') as in_pickle:
    machine = pickle.load(in_pickle)

## Retrieve and store all descendant and siblings for machine senses

In [ ]:
get_semantic_class_idx = lambda data: [l[-1] for s in data['data']['senses']  for l in s['semantic_class_ids']]

# probably better make a normal function for this
get_branches = lambda sc_idx: { idx : query_oed(auth, 'semanticclass', idx, 
                                              level='branchsenses',
                                              flags="current_in='1750-1950'",
                                              verbose=False
                                               ) 
                                                   for idx in tqdm(sc_idx) 
                                      }





In [ ]:
sem_class_idx = get_semantic_class_idx(machine)

In [ ]:
branches = get_branches(sem_class_idx)

In [ ]:
# with open('./dev/machine_sc_branches.pckl','wb') as out_pickle:
#     pickle.dump(branches,out_pickle)

In [ ]:
with open('./dev/machine_sc_branches.pckl','rb') as in_pickle:
    branches = pickle.load(in_pickle)

In [ ]:
branches['120172']['data']